In [0]:
spark

In [0]:
# Load the Dataset
import pandas as pd
pdf = pd.read_excel("/Workspace/Users/nikhilpmarihal@gmail.com/ecommerce_data.xlsx")
df = spark.createDataFrame(pdf)

display(df)

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2019-10-06T02:05:00.000Z,view,75939936,2668875176,furniture.living_room.sofa,shiseido,1321.53,512585723,7199665a67cb3
2019-11-28T12:39:00.000Z,view,97040461,2131004561,appliances.environment.water_heater,shiseido,350.96,555026779,1c54eb341aae8d
2019-11-03T15:18:00.000Z,purchase,69607165,2554264939,computers.notebook,lenovo,303.15,562399585,1f708934878dc1
2019-11-13T19:39:00.000Z,view,68280751,2991571625,electronics.smartphone,lenovo,1754.44,583369254,22f9da9f3ea165
2019-10-19T05:56:00.000Z,view,93672506,2770332132,appliances.environment.water_heater,shiseido,31.51,574123441,21da49ede4756b
2019-11-13T15:08:00.000Z,view,68549263,2778999088,furniture.living_room.sofa,null,326.33,530972953,3a0679951a99f
2019-11-16T14:46:00.000Z,cart,66264481,2519108358,electronics.smartphone,aqua,1625.21,523358267,1144504a96413a
2019-10-01T03:58:00.000Z,purchase,62118102,2104194919,computers.notebook,apple,1805.36,507366315,21d5abd92a9c2e
2019-12-03T07:45:00.000Z,view,28519606,2271236488,null,lenovo,1365.46,575094204,1e8ff0b1b073ec
2019-11-13T14:43:00.000Z,view,38523169,2392556245,computers.notebook,creed,1672.78,537623005,15c56a5ca40f19


In [0]:
# 2_Basic operations to see columns - product_id, brand and price.
df.select("product_id", "brand", "price").show(10)

+----------+--------+-------+
|product_id|   brand|  price|
+----------+--------+-------+
|  75939936|shiseido|1321.53|
|  97040461|shiseido| 350.96|
|  69607165|  lenovo| 303.15|
|  68280751|  lenovo|1754.44|
|  93672506|shiseido|  31.51|
|  68549263|    NULL| 326.33|
|  66264481|    aqua|1625.21|
|  62118102|   apple|1805.36|
|  28519606|  lenovo|1365.46|
|  38523169|   creed|1672.78|
+----------+--------+-------+
only showing top 10 rows


In [0]:
# 3_Filter by price greater than 100.
df.filter(df.price > 500).count()

747

In [0]:
# 4_Count of types of events.
df.groupBy("event_type").count().show()

+----------+-----+
|event_type|count|
+----------+-----+
|      view|  361|
|  purchase|  345|
|      cart|  294|
+----------+-----+



In [0]:
# 5_Count of top brands.
from pyspark.sql.functions import desc

top_brands = (
    df.groupBy("brand")
      .count()
      .orderBy(desc("count"))
      .limit(5)
)

top_brands.show()


+--------+-----+
|   brand|count|
+--------+-----+
|   creed|  150|
|luminarc|  148|
|    NULL|  145|
|  lenovo|  145|
|   apple|  141|
+--------+-----+

